In [61]:
import sys
from IPython.display import clear_output
import torch
from torch import nn
from torch.nn import functional as F

# Find a way to deal with this better (importing utils)
sys.path.append('../')
from utils.util import MatchData

# Use CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == 'cuda':
    print(f'Using cuda device {torch.cuda.get_device_name()}')
else:
    print(f"Using {device} device")

Using cuda device NVIDIA GeForce RTX 3060 Laptop GPU


In [62]:
# Load data    
load_path = '../../data/processed/processed_match_data_euw1_1667433600_1668520576.csv'
data = MatchData(load_path, device = device, data_format ='champ', sequential = True)
(X_tr, Y_tr), (X_te, Y_te) = data.test_train_split(split_size=0.8, seed=42)
batches = data.create_mini_batches(batch_size = 32)

In [63]:
# RNN module

class RnnModel(nn.Module):
    def __init__(self, data, embed_dim:int, rnn_hidden_size:int):
        super(RnnModel, self).__init__()
        
        self.embed_dim = embed_dim
        self.rnn_hidden_size = rnn_hidden_size
        
        self.embed = nn.Embedding(data.num_champs, embed_dim)
        self.rnn = nn.RNN(input_size=2*embed_dim, hidden_size = rnn_hidden_size,
                          batch_first = True, bidirectional = False)
        self.linear = nn.Linear(rnn_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    
    def forward(self, x):
        # x.shape = (batch_size, 10)
        x_emb = self.embed(x) # x_emb.shape = (batch_size, 10, embed_dim)
        x_flatten = x_emb.reshape(-1, 5, 2*self.embed_dim) # x_flatten.shape = (batch_size, 5, 2*embed_dim)
        h_states = self.rnn(x_flatten)[0] # h_states.shape = (batch_size, 5, rnn_hidden_size)
        h_avg = h_states.mean(dim=1) # Take avg?
        logits = self.linear(h_avg)
        probs = self.sigmoid(logits)
        
        return probs
        

In [64]:
# Hyper parameters & initialisation 

embed_size = 3 # Embedding size
rnn_hidden_size = 5 # Hidden size

learning_rate = 0.01 # Learning rate
weight_decay = 0.0005 # wd

# Model
model = RnnModel(data, embed_size, rnn_hidden_size).to(device)
print(f'{sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters.')
print(model)

# Loss function and optimiser
loss_fn = nn.BCELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

560 parameters.
RnnModel(
  (embed): Embedding(163, 3)
  (rnn): RNN(6, 5, batch_first=True)
  (linear): Linear(in_features=5, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [65]:
# Train functions

@torch.no_grad()
def test(x, y):
    """Returns loss and accuracy"""
    
    model.eval()
    y_pred = model.forward(x)
    loss = round(loss_fn(y_pred, y).item(), 4)
    accuracy = round((y_pred.round() == y).sum().item() / y.shape[0], 4)
    
    return loss, accuracy


def train(n_epochs):
    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1} / {n_epochs} complete:')
    
        train_loss, train_acc = test(X_tr, Y_tr)
        test_loss, test_acc = test(X_te, Y_te)
        print(f'Train loss = {train_loss}. Train accuracy = {train_acc}.')
        print(f'Test loss = {test_loss}. Test accuracy = {test_acc}.')
        
        train_epoch()
        clear_output(wait=True)


def train_epoch():
    model.train()
    for i, (X_b, Y_b) in enumerate(batches):
        
        Y_pred = model.forward(X_b)
        loss = loss_fn(Y_pred, Y_b)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

In [66]:
# Training
train(200)

Epoch 2 / 200 complete:
Train loss = 0.6932. Train accuracy = 0.5023.
Test loss = 0.6945. Test accuracy = 0.4877.
